## Splits data into ones with 1 planet and 2 planets respectively.

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# Import cleaned/merged dataset
FILENAME = r'data_cleaned\merged_GLSfitted.csv'
df = pd.read_csv(FILENAME)
# Drop unused columns
df = df.drop(columns=['pl_letter', 'mass_sini', 'mass_sini_error', 'radius', 'radius_error'])

df.head(3)

,name_exoclock+holczer,name_exoplanet.eu,star_name,T0_(BJD_TDB),T0_unc.,P_(days),P_unc.,mass,mass_error,orbital_period,...,Attv2,Nbest,scatter,bic0,bic1,bic2,delta_bic,fap,sampled_f,TTV_strength
0,NaN,55 Cnc Ab,55 Cnc A,NaN,NaN,NaN,NaN,0.8400,0.2300,14.65314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,55 Cnc Ac,55 Cnc A,NaN,NaN,NaN,NaN,0.1714,0.0055,44.37300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,55 Cnc Ad,55 Cnc A,NaN,NaN,NaN,NaN,3.8780,0.0680,5574.20000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Mask by planet_count = 1
df_1p = df[df.planet_count == 1]
df_1p = df_1p.drop(columns=['orbital_period', 'orbital_period_error'])
EXPORT_FILENAME = r'data_cleaned\merged_1planet.csv'
df_1p.to_csv(EXPORT_FILENAME, index=False)
df_1p.head(3)

,name_exoclock+holczer,name_exoplanet.eu,star_name,T0_(BJD_TDB),T0_unc.,P_(days),P_unc.,mass,mass_error,eccentricity,...,Attv2,Nbest,scatter,bic0,bic1,bic2,delta_bic,fap,sampled_f,TTV_strength
5,CoRoT-1b,CoRoT-1 b,CoRoT-1,2.455112e+06,0.000067,1.508968,5.800000e-08,1.03,0.12,0.071,...,NaN,1.0,0.558962,88.844391,73.508699,79.693882,15.335692,0.015197,19395.0,no_TTV
6,CoRoT-11b,CoRoT-11 b,CoRoT-11,2.456068e+06,0.000340,2.994278,4.500000e-07,2.33,0.27,0.070,...,0.002477,2.0,0.565121,34.474703,25.756732,25.491845,8.982858,NaN,9174.0,no_TTV
7,CoRoT-19b,CoRoT-19 b,CoRoT-19,2.456551e+06,0.000450,3.897138,8.800000e-07,1.11,0.06,0.047,...,NaN,1.0,0.370604,12.502658,14.317067,20.677455,-1.814409,1.000000,6455.0,no_TTV


In [9]:
df_2p = df[df.planet_count == 2]

# Sort by ascending star_name, descending period
df_2p = df_2p.sort_values(by=['star_name', 'orbital_period'], ascending=[True, False])
# Assumes exactly 2 rows per star_name. Higher period → outer, lower → inner.
df_2p['planet_pos'] = (
    df_2p.groupby('star_name')
      .cumcount()
      .map({0: 'outer', 1: 'inner'})
)

# Get period ratio P2/P1
df_2p['period_ratio'] = (
    df_2p.pivot(index='star_name', columns='planet_pos', values='orbital_period')
      .eval('outer / inner')
      .reindex(df_2p['star_name'])
      .to_numpy()
)

# Find approximate MMR
allowed_MMR = {
    '2:1': 2/1, '3:2': 3/2, '4:3': 4/3, '5:4': 5/4, '6:5': 6/5, '7:6': 7/6,
    '3:1': 3/1, '5:3': 5/3, '7:5': 7/5,
    '4:1': 4/1, '5:2': 5/2,
    '5:1': 5/1, '7:3': 7/3}
vals = np.array(list(allowed_MMR.values()))
keys = np.array(list(allowed_MMR.keys()))

def closest_allowed_MMR(x):
    if np.isnan(x) or x >= 4.1:
        return np.nan
    return keys[np.argmin(np.abs(vals - x))]

df_2p['MMR'] = df_2p['period_ratio'].apply(closest_allowed_MMR)

# Get j and N
from natsume.common import get_MMR
df_2p[['j', 'N']] = (
    df_2p['MMR']
      .apply(lambda x: pd.Series(get_MMR(x)) if isinstance(x, str) else pd.Series([np.nan, np.nan]))
)
# Get Delta
from natsume.common import get_NormalizedResonanceDistance
df_2p['Delta'] = get_NormalizedResonanceDistance(1, df_2p.period_ratio, df_2p.j, df_2p.N)

EXPORT_FILENAME = r'data_cleaned\merged_2planet.csv'
df_2p.to_csv(EXPORT_FILENAME, index=False)
df_2p.head(3)

,name_exoclock+holczer,name_exoplanet.eu,star_name,T0_(BJD_TDB),T0_unc.,P_(days),P_unc.,mass,mass_error,orbital_period,...,delta_bic,fap,sampled_f,TTV_strength,planet_pos,period_ratio,MMR,j,N,Delta
13,NaN,GJ 1132 c,GJ 1132,NaN,NaN,NaN,NaN,0.008314,0.001381,8.929000,...,NaN,NaN,NaN,NaN,outer,5.481509,NaN,NaN,NaN,NaN
12,GJ1132b,GJ 1132 b,GJ 1132,2.458041e+06,0.000093,1.62893,8.600000e-07,0.005220,0.000720,1.628931,...,33.734042,0.000001,8764.0,no_TTV,inner,5.481509,NaN,NaN,NaN,NaN
32,NaN,HAT-P-11 c,HAT-P-11,NaN,NaN,NaN,NaN,3.060000,0.400000,3299.000000,...,NaN,NaN,NaN,NaN,outer,674.945446,NaN,NaN,NaN,NaN
